
Para eso uso el siguiente modelo:

$$Y(i,t) = \alpha_i + \beta \cdot t + \gamma_{inicial} \Theta_{inicial} + \gamma_{final} \Theta_{final}$$

Donde i representa el sujeto, t representa la sesion (aca se juntan una sesion del test, las doce de entrenamiento (separanto los niveles dentro de cada sesion) y el test final (esto habria que revisarlo si conviene o no).  

Como esta regresion lineal es un lio de definir, la idea es hacer un "stack" donde se transforme todo en un sistema de matrices con la forma:

$$ \bar{Y} = \bar{\beta} \cdot \bar{\bar{X}}$$ 

Donde si i va de 1 a 3 y t de 1 a 4 queda escrito de la forma :

$$ X = 
 \begin{pmatrix}
  \beta & \alpha_1 & \alpha_2 & \alpha_3 & \gamma_{inicial} & \gamma_{final} \\
  0 & 1 & 0 & 0 & 1 & 0\\
  1 & 1 & 0 & 0 & 0 & 0\\
  2 & 1 & 0 & 0 & 0 & 0\\
  3 & 1 & 0 & 0 & 0 & 1\\
  0 & 0 & 1 & 0 & 1 & 0\\
  1 & 0 & 1 & 0 & 0 & 0\\
  2 & 0 & 1 & 0 & 0 & 0\\
  3 & 0 & 1 & 0 & 0 & 1\\
  0 & 0 & 0 & 1 & 1 & 0\\
  1 & 0 & 0 & 1 & 0 & 0\\
  2 & 0 & 0 & 1 & 0 & 0\\
  3 & 0 & 0 & 1 & 0 & 1\\
 \end{pmatrix}
$$

Una vez generada esa matriz usamos el codigo que figura aca:

http://stackoverflow.com/questions/24393518/python-sklearn-linear-model-linearregression-working-weird

In [29]:
# Cargamos los scripts usuales
%matplotlib inline
%load_ext autoreload
%autoreload 2
from scripts.db import *
from scripts.plots import *
from scripts.stats import *
from scripts.statsTransferencia import *
import numpy as np
import statsmodels.api as sm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
#Cargamos los datos y los ordenamos en una dataframe (ojo que lo retoque del original para que ordene las columnas como corresponde)

import pandas as pd

resumen = dataNumerica()
data = pd.DataFrame.from_dict(resumen, orient='index')

#Trasnformamos los valores que interesan en columnas (Por ahora voy a hacer todo en angulos)
entrenamientoP = pd.DataFrame()
entrenamientoA = pd.DataFrame()
entrenamiento = pd.DataFrame()

entrenamientoP['TI'] = data[data['Orientacion'] == 'P30']['TestInicialAngulos'].map(lambda x: x[0])
entrenamientoA['TI'] = data[data['Orientacion'] == 'A30']['TestInicialAngulos'].map(lambda x: x[4])

# entrenamiento['TI'] = data[data['Orientacion'] != 'CONTROL']['TestInicialAngulos'].map(lambda x: x[0])
# entrenamiento['TI'] = data[data['Orientacion'] == 'A30']['TestInicialAngulos'].map(lambda x: x[4])
# entrenamiento['TF'] = data[data['Orientacion'] != 'CONTROL']['TestFinalAngulos'].map(lambda x: x[0])
# entrenamiento['TF'] = data[data['Orientacion'] == 'A30']['TestFinalAngulos'].map(lambda x: x[4])


orientaciones = {0:'EntrenamientoInicialEnAngulos',1:'EntrenamientoMedioEnAngulos',2:'EntrenamientoFinalEnAngulos'}

for i in range(4):
    for key, value in orientaciones.items():
        entrenamientoP['E'+str(key)+'D'+str(i)] = data[data['Orientacion'] == 'P30'][orientaciones[key]].map(lambda x: x[i])
        entrenamientoA['E'+str(key)+'D'+str(i)] = data[data['Orientacion'] == 'A30'][orientaciones[key]].map(lambda x: x[i])
        entrenamiento['E'+str(key)+'D'+str(i)] = data[data['Orientacion'] != 'CONTROL'][orientaciones[key]].map(lambda x: x[i])

entrenamientoP['TF'] = data[data['Orientacion'] == 'P30']['TestFinalAngulos'].map(lambda x: x[0])
entrenamientoA['TF'] = data[data['Orientacion'] == 'A30']['TestFinalAngulos'].map(lambda x: x[4])

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:536: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:508: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:510: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


In [48]:
# Armamos las matrices de ajuste segun corresponda, despues se eliminan los datos invalidos

# los datos vienen agrupados en el siguiente orden: primero todos los correspondientes a un mismo sujeto, luego se respeta el orden, es decir se agrupa por dia, y va primero el test inicial y al final el final, en caso de que esten incluidos
sujetos = 5 # Numeros de sujetos entrenados
dias = 4 # Numero de dias que vino casa sujeto a entrenar 
sesionesTotales = dias * sujetos # Se llama sesion a cada vez que viene un sujeto
nivelesXsesion = 3 # Numero de niveles que hay en cada sesion
nivelesXusuario = dias * nivelesXsesion # Numero total de niveles que hace cada usuario

incluirExtremos = False # Marca si se agrega el test inicial y final a la serie de datos

if incluirExtremos:
    nivelesXusuario = nivelesXusuario + 2

# Estos vectores son los vectores que ajustan el beta que representa la pendiente
betaslocal = [i for i in range(nivelesXsesion)] * sesionesTotales
betasglobales = [i for i in range(nivelesXusuario)] * sujetos

# Alpha representa la matriz que agrupa por usuario
alphas = [[]*(sujetos)]*(sujetos)

for sujetox in range(sujetos):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * nivelesXusuario
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * nivelesXusuario

            
# La matriz deta marca todos los datos que corresponden a un mismo dia            

detas = [[]*dias]*dias

for sujeto in range(sujetos):
    for diax in range (dias):
        if incluirExtremos:
            detas[diax] = detas[diax] + [0]
        for diay in range (dias):
            if diax==diay:
                detas[diax] = detas[diax] + [1] * nivelesXsesion
            else:
                detas[diax] = detas[diax] + [0] * nivelesXsesion
        if incluirExtremos:
            detas[diax] = detas[diax] + [0]

gamma1 = ([1] + [0] * (nivelesXusuario -1)) * sujetos
gamma2 = ([0] * (nivelesXusuario -1) + [1]) * sujetos

# La matriz que separa los datos del segundo dia.
feed = [0,1,0] * sesionesTotales

# buscamos la matriz de datos

if incluirExtremos:
    entrenamientoAProcesar = pd.concat([entrenamientoA,entrenamientoP])
else:
    entrenamientoAAProcesar = entrenamientoA.drop(['TI','TF'],1)
    entrenamientoPAProcesar = entrenamientoP.drop(['TI','TF'],1)
    entrenamientoAProcesar = entrenamiento

entrenamientoAProcesar = entrenamientoPAProcesar

secuencia = entrenamientoAProcesar.stack()
valores = secuencia.tolist()

# Nos fijamos si hay que corregir la sesion que no va
sacarPendiente = True

if sacarPendiente:
    for i, value in enumerate(valores):
        if value>50:
            if incluirExtremos:
                if gamma1[i] == 0:
                    if gamma2[i] ==0:
                        valores[i] = 0
                        valores[i+1] = 0
                        valores[i+2] = 0
                        break
            else:
                valores[i] = 0
                valores[i+1] = 0
                valores[i+2] = 0
                break

sacarusuario1 = False

if sacarusuario1:
    valores[0:(nivelesXusuario-1)] = [0]*nivelesXusuario
                
sacarDia1 = False

if sacarDia1:
    for i in range(nivelesXusuario*sujetos):
        if incluirExtremos:
            if i%nivelesXusuario in [1,2,3]:
                valores[i]=0
        else:
            if i%nivelesXusuario in [0,1,2]:
                valores[i]=0

display (entrenamientoAProcesar)

# Armamos la matriz completa
unos = [1] * len(valores)
# matriz = [unos] + [betaslocal] + [betasglobales] + alphas + [gamma1] + [gamma2] + detas + [valores]

# elegimops que queremos ajustar
matriz = [unos] + [betaslocal] + [feed] + [valores]


matriz = list(map(list, zip(*matriz)))

# Limpiamos la matriz de las filas que no van
matriz = [item for item in matriz if item[-1]]

print (np.matrix(matriz))
# Hacemos el ajuste

matrizPura = [item[:-1] for item in matriz]
valores = [item[-1] for item in matriz]


x = np.array(matrizPura)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

,E0D0,E1D0,E2D0,E0D1,E1D1,E2D1,E0D2,E1D2,E2D2,E0D3,E1D3,E2D3
Sujeto03,21.5,18.5,19.0,34.0,26,13.0,23,15,18.0,18.5,24.5,10.0
Sujeto04,17.5,20.0,8.5,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0
Sujeto05,24.0,12.5,16.5,15.5,15,12.5,13,9,7.5,10.0,15.0,10.5


[[  1.    0.    0.   21.5]
 [  1.    1.    1.   18.5]
 [  1.    2.    0.   19. ]
 [  1.    0.    0.   34. ]
 [  1.    1.    1.   26. ]
 [  1.    2.    0.   13. ]
 [  1.    0.    0.   23. ]
 [  1.    1.    1.   15. ]
 [  1.    2.    0.   18. ]
 [  1.    0.    0.   18.5]
 [  1.    1.    1.   24.5]
 [  1.    2.    0.   10. ]
 [  1.    0.    0.   17.5]
 [  1.    1.    1.   20. ]
 [  1.    2.    0.    8.5]
 [  1.    0.    0.   24. ]
 [  1.    1.    1.   12.5]
 [  1.    2.    0.   16.5]
 [  1.    0.    0.   15.5]
 [  1.    1.    1.   15. ]
 [  1.    2.    0.   12.5]
 [  1.    0.    0.   13. ]
 [  1.    1.    1.    9. ]
 [  1.    2.    0.    7.5]
 [  1.    0.    0.   10. ]
 [  1.    1.    1.   15. ]
 [  1.    2.    0.   10.5]]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squ

In [28]:
display (entrenamientoA)

,E0D0,E1D0,E2D0,E0D1,E1D1,E2D1,E0D2,E1D2,E2D2,E0D3,E1D3,E2D3
Sujeto01,65.6,39.6,24.4,5.6,16,14.4,12.0,19.2,14.4,16.0,17.2,19.2
Sujeto02,20.0,15.6,16.4,16.8,16,13.6,13.6,14.8,19.2,19.6,20.8,16.4
